In [1]:
pip install gensim


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for fst-pso: filename=fst_pso-1.8.1-py3-none-any.whl size=20430 sha256=dbdac89135f4bf9439015e18ac433cfb6df75ffa245823bd33210b9f3b2ddfec
  Stored in directory: /Users/tahora/Library/Caches/pip/wheels/69/f5/e5/18ad53fe1ed6b2af9fad05ec052e4acbac8e92441df44bad2e
  Created wheel for miniful: filename=miniful-0.0.6-py3-none-any.whl size=3513 sha256=11db1bd203062a94be70287b6a7ec580c70d9fd14507ca9092f76749e0f86bba
  Stored in directory: /Users/tahora/Library/Caches/pip/wheels/9d/ff/2f/afe4cd56f47de147407705626517d68bea0f3b74eb1fb168e6
Successfully built fst-pso miniful
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 6.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
import re
from tqdm.notebook import tqdm
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras.utils import plot_model
import tensorflow as tf
import random
from gensim.models import Word2Vec
from transformers import BertTokenizer, TFBertModel
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

# Define the mispell_dict for text correction
mispell_dict = {
    "ain't": "is not",
    "cannot": "can not",
    "aren't": "are not",
    "can't": "can not",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'll": "he will",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "I'd": "I would",
    "I'd've": "I would have",
    "I'll": "I will",
    "I'll've": "I will have",
    "I'm": "I am",
    "I've": "I have",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "this's": "this is",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "here's": "here is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "wont": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have",
    'colour': 'color',
    'centre': 'center',
    'favourite': 'favorite',
    'travelling': 'traveling',
    'counselling': 'counseling',
    'theatre': 'theater',
    'cancelled': 'canceled',
    'labour': 'labor',
    'organisation': 'organization',
    'wwii': 'world war 2',
    'citicise': 'criticize',
    'youtu ': 'youtube ',
    'Qoura': 'Quora',
    'sallary': 'salary',
    'Whta': 'What',
    'narcisist': 'narcissist',
    'howdo': 'how do',
    'whatare': 'what are',
    'howcan': 'how can',
    'howmuch': 'how much',
    'howmany': 'how many',
    'whydo': 'why do',
    'doI': 'do I',
    'theBest': 'the best',
    'howdoes': 'how does',
    'Etherium': 'Ethereum',
    'narcissit': 'narcissist',
    'bigdata': 'big data',
    '2k17': '2017',
    '2k18': '2018',
    'qouta': 'quota',
    'exboyfriend': 'ex boyfriend',
    'airhostess': 'air hostess',
    "whst": 'what',
    'watsapp': 'whatsapp',
    'demonitisation': 'demonetization',
    'demonitization': 'demonetization',
    'demonetisation': 'demonetization'
}
mispell_dict = {k.lower(): v.lower() for k, v in mispell_dict.items()}

# Load the dataset
data = pd.read_csv('file:///Users/Tahora/Downloads/train-balanced-sarcasm.csv')

# Filter relevant columns
data = data[['labels', 'comments']]

# Function for text cleaning and correction
def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Correct misspelled words
    text = ' '.join([mispell_dict.get(word, word) for word in text.split()])
    
    # Remove special characters and extra whitespace
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Apply text cleaning to the 'comments' column
data['comments'] = data['comments'].apply(clean_text)

# Split the data into train, validation, and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

# Tokenize the text data
max_features = 10000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(train_data['comments'])

train_sequences = tokenizer.texts_to_sequences(train_data['comments'])
val_sequences = tokenizer.texts_to_sequences(val_data['comments'])
test_sequences = tokenizer.texts_to_sequences(test_data['comments'])

# Pad the sequences
max_sequence_length = 100
train_sequences = pad_sequences(train_sequences, maxlen=max_sequence_length)
val_sequences = pad_sequences(val_sequences, maxlen=max_sequence_length)
test_sequences = pad_sequences(test_sequences, maxlen=max_sequence_length)

# Convert labels to binary format
train_labels = train_data['labels'].astype(int)
val_labels = val_data['labels'].astype(int)
test_labels = test_data['labels'].astype(int)

# Word2Vec word embeddings
embedding_dim = 100
word2vec_model = Word2Vec(sentences=train_data['comments'], vector_size=embedding_dim, min_count=1)
word2vec_embeddings = np.zeros((max_features, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i < max_features:
        if word in word2vec_model.wv:
            word2vec_embeddings[i] = word2vec_model.wv[word]

# BERT embeddings
tokenizer_bert = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

train_sequences_bert = tokenizer_bert.batch_encode_plus(
    train_data['comments'].tolist(),
    truncation=True,
    max_length=max_sequence_length,
    padding='max_length',
    return_tensors='tf'
)
val_sequences_bert = tokenizer_bert.batch_encode_plus(
    val_data['comments'].tolist(),
    truncation=True,
    max_length=max_sequence_length,
    padding='max_length',
    return_tensors='tf'
)
test_sequences_bert = tokenizer_bert.batch_encode_plus(
    test_data['comments'].tolist(),
    truncation=True,
    max_length=max_sequence_length,
    padding='max_length',
    return_tensors='tf'
)

train_input_ids = train_sequences_bert['input_ids']
train_attention_masks = train_sequences_bert['attention_mask']
val_input_ids = val_sequences_bert['input_ids']
val_attention_masks = val_sequences_bert['attention_mask']
test_input_ids = test_sequences_bert['input_ids']
test_attention_masks = test_sequences_bert['attention_mask']

train_embeddings_bert = bert_model([train_input_ids, train_attention_masks])[0]
val_embeddings_bert = bert_model([val_input_ids, val_attention_masks])[0]
test_embeddings_bert = bert_model([test_input_ids, test_attention_masks])[0]

# Model selection
inputs = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(max_features, embedding_dim, weights=[word2vec_embeddings], trainable=False)(inputs)
lstm_layer = LSTM(128, return_sequences=True)(embedding_layer)
global_max_pooling_layer = GlobalMaxPool1D()(lstm_layer)
dropout_layer = Dropout(0.2)(global_max_pooling_layer)
dense_layer = Dense(64, activation='relu')(dropout_layer)
outputs = Dense(1, activation='sigmoid')(dense_layer)
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Grid search for hyperparameter tuning
parameters = {'batch_size': [32, 64], 'epochs': [5, 10]}
grid_search = GridSearchCV(estimator=model, param_grid=parameters, scoring='accuracy', cv=3)
grid_search.fit(train_sequences, train_labels)

# Get the best model from grid search
best_model = grid_search.best_estimator_

# Model training
best_model.fit(train_sequences, train_labels, batch_size=best_model.batch_size, epochs=best_model.epochs, validation_data=(val_sequences, val_labels))

# Model evaluation
test_predictions = best_model.predict(test_sequences)
test_predictions = (test_predictions > 0.5).astype(int)

accuracy = accuracy_score(test_labels, test_predictions)
precision = precision_score(test_labels, test_predictions)
recall = recall_score(test_labels, test_predictions)
f1 = f1_score(test_labels, test_predictions)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)
